In [2]:
from warnings import filterwarnings

filterwarnings("ignore")

# Data loading and pre-processing
from keras.datasets import cifar10
from keras.utils import np_utils

# Model related imports
import tensorflow as tf
import keras

from keras.optimizers import Adam

from utils.hyperparams import parse_args
from utils.utils import create_dir_if_not_exists, save_loss_plots
from utils.model import get_model, convert_to_masked_model


import os
import csv

ModuleNotFoundError: No module named 'pruned_layers'

In [3]:
import numpy as np
# Read Data and do initial pre-processing
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# convert to float32 (to not make all features zero)
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

# divide features by 255 so that pixels have value in the range of [0,1]
x_train /= 255
x_test /= 255

x_train = 2 * x_train - 1
x_test = 2 * x_test - 1

# convert class vectors to binary class matrices (one-hot encoding)
n_output = 10
y_train = np_utils.to_categorical(y_train, n_output)
y_test = np_utils.to_categorical(y_test, n_output)

print("X_Train: {}\nX_Test:  {}\nY_Train: {}\nY_Test:  {}" \
      .format(x_train.shape, x_test.shape, y_train.shape, y_test.shape))
print("Train Samples: {}, Test Samples: {}".format(x_train.shape[0], x_test.shape[0]))

X_Train: (50000, 32, 32, 3)
X_Test:  (10000, 32, 32, 3)
Y_Train: (50000, 10)
Y_Test:  (10000, 10)
Train Samples: 50000, Test Samples: 10000


In [4]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, Lambda, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.layers.normalization import BatchNormalization

In [5]:
from utils.pruned_layers import *

model = Sequential()
model.add(pruned_Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='valid', input_shape=[32, 32, 3]))
model.add(MaxPooling2D(8,8))
model.add(Flatten())
model.add(Dense(units=10, activation='softmax'))

optimizer = Adam(lr=0.001, decay=1e-6)
# Model compilation with sparse categorical crossentropy loss
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
tracking <tf.Variable 'pruned__conv2d_1/bias:0' shape=(64,) dtype=float32> bias


In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
pruned__conv2d_1 (pruned_Con (None, 32, 32, 64)        3520      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 13,770
Trainable params: 12,042
Non-trainable params: 1,728
_________________________________________________________________


In [7]:
model.layers[0].get_weights()[2]

array([[[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]],


       [[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]],


       [[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],

        [[1., 1., 1.

In [ ]:
history = model.fit(x_train, y_train, batch_asize=100, epochs=1,validation_data=(x_test, y_test))

In [ ]:
a = model.layers[0].get_weights()[0].shape

In [ ]:
model.layers[0].set_mask(np.ones(a))

In [ ]:
model.layers[0].set_mask(np.ones(model.layers[0].get_weights()[0].shape))

In [ ]:
np.abs(model.layers[1].get_weights()[0])> np.percentile(np.abs(model.layers[1].get_weights()[0]),55)

In [ ]:
# Instantiate Model
model = get_model(arg)
# Load pre-trained model
model.load_weights(arg.weights_path)

optimizer = Adam(lr=arg.lr, decay=1e-6)

# Model compilation with sparse categorical crossentropy loss
model.compile(#loss='sparse_categorical_crossentropy',
              loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

batch_size = arg.batch_size
training = arg.training
compressing = arg.compressing

# make sure weights are loaded correctly by evaluating the model here and printing the output

test_loss, test_acc = model.evaluate(x_test, y_test, batch_size=batch_size)


# convert the layers to maskable_layers:
prunable_model = convert_to_masked_model(model, arg.pruning_percentage)

optimizer = keras.optimizers.Adam(lr=arg.lr, decay=1e-6)
# now complie the prunable model with sparse categorical crossentropy loss function as you did in part 1
prunable_model.compile(loss='sparse_categorical_crossentropy',
                       optimizer=optimizer,
                       metrics=['accuracy'])

# make sure weights are loaded correctly by evaluating the prunable model here and printing the output

#
experiment_name = "Training_{}_Compressing_{}_Layer_{}_percentage_{}".format(arg.Training, arg.Compressing, arg.Layer, arg.pruning_percentage)

create_dir_if_not_exists('./results')
create_dir_if_not_exists(os.path.join('./results', experiment_name))
# save_loss_plots(history, os.path.join('./results', experiment_name))

with open(os.path.join('./results', experiment_name, 'results.csv'), 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter='\t')
    writer.writerow(['pruning rate', 'accuracy'])
